# 7_SCIKIT-LEARN

## 7.1. Основы работы с классами, строящими модели предварительной подготовки данных и модели машинного обучения

Scikit-learn использует объектно-ориентированный подход, где каждая модель — это класс с определенными методами. Давайте разберемся, как это работает.

**Классификаторы vs Регрессоры**  
- Классификаторы: `[Метод]Classifier` (например, `RandomForestClassifier`)
- Регрессоры: `[Метод]Regressor` (например, `LinearRegression`)
- Универсальные: `[Метод]` (например, `KMeans` для кластеризации)

Ключевые принципы
1. Единообразие интерфейса: Все модели имеют методы .fit(), .predict()
2. Независимость от типа данных: Работают с NumPy arrays и pandas DataFrames
3. Совместимость: Можно комбинировать в конвейерах (pipelines)
4. Наследование: Используйте BaseEstimator и TransformerMixin для совместимости

Важные методы
|Метод|Назначение|Возвращает|
|-|-|-|
|`.fit()`|Обучение модели|`self`|
|`.predict()`|Предсказания|Массив предсказаний|
|`.transform()`|Преобразование данных|Преобразованные данные|
|`.fit_transform()`|Обучение + преобразование|Преобразованные данные|
|`.score()`|Оценка качества|Значение метрики|

**Стандартный workflow в scikit-learn**

```python
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

# 1. Создание экземпляра класса
model = RandomForestClassifier()
scaler = StandardScaler()

# 2. Обучение модели
scaler.fit(X_train)          # Только признаки
model.fit(X_train, y_train)  # Признаки + метки

# 3. Применение модели
X_scaled = scaler.transform(X_test)
predictions = model.predict(X_scaled)
```

**Создание собственного класса преобразователя**

```python
# Базовый шаблон
from sklearn.base import BaseEstimator, TransformerMixin

class CustomTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, param1=default_value):
        self.param1 = param1
    
    def fit(self, X, y=None):
        # Вычисление параметров
        return self
    
    def transform(self, X):
        # Применение преобразования
        return X
```

**Пример: Замена пропусков средним (MeanImputer)**

In [8]:
import pandas as pd
import numpy as np

class MeanImputer:
    def __init__(self, copy=True):
        self.copy = copy
        self._encoder_dict = {}
    
    def _is_numpy(self, X):
        return isinstance(X, np.ndarray)
    
    def fit(self, X, y=None):
        # Создаем словарь для хранения средних значений
        self._encoder_dict = {}
        
        # Проверяем тип данных
        is_np = self._is_numpy(X)
        
        # Обработка 1D массива
        if len(X.shape) == 1:
            X = X.reshape(-1, 1)
        
        # Вычисляем средние значения
        if is_np:
            for col in range(X.shape[1]):
                self._encoder_dict[col] = np.nanmean(X[:, col])
        else:
            for col in X.columns:
                self._encoder_dict[col] = X[col].mean()
        
        return self
    
    def transform(self, X):
        if self.copy:
            X = X.copy()
        
        is_np = self._is_numpy(X)
        
        if len(X.shape) == 1:
            X = X.reshape(-1, 1)
        
        # Замена пропусков
        if is_np:
            for col in range(X.shape[1]):
                X[:, col] = np.nan_to_num(X[:, col], nan=self._encoder_dict[col])
        else:
            for col in X.columns:
                X[col] = np.where(X[col].isnull(), self._encoder_dict[col], X[col])
        
        return X

In [9]:
# Использование MeanImputer

# Пример с DataFrame
toy_train = pd.DataFrame({
    'Balance': [8.3, np.nan, 10.2, 3.1],
    'Age': [23, 29, 36, np.nan]
})

# Создание и применение преобразователя
imp = MeanImputer()
imp.fit(toy_train)
toy_train_inputed = imp.transform(toy_train)
print(toy_train_inputed)

   Balance        Age
0      8.3  23.000000
1      7.2  29.000000
2     10.2  36.000000
3      3.1  29.333333


**Создание собственной модели машинного обучения**

Пример: K-Nearest Neighbors (KNN)

In [10]:
import math

class KNN_Estimator:
    def __init__(self, k=5, task='classification'):
        self.k = k
        self.task = task
    
    def _euclidean_distance(self, x1, x2):
        """Вычисляет евклидово расстояние между двумя точками"""
        distance = 0
        for i in range(len(x1)):
            distance += (x1[i] - x2[i]) ** 2
        return math.sqrt(distance)
    
    def _vote(self, neighbor_labels):
        """Голосование для классификации"""
        counts = np.bincount(neighbor_labels.astype('int'))
        return counts.argmax()
    
    def fit(self, X, y):
        """Запоминаем обучающие данные"""
        self.X_train = X
        self.y_train = y
        return self
    
    def predict(self, X):
        """Предсказание для новых данных"""
        predictions = []
        
        for test_sample in X:
            # Находим расстояния до всех точек
            distances = [self._euclidean_distance(test_sample, x) 
                        for x in self.X_train]
            
            # Находим k ближайших соседей
            nearest_indices = np.argsort(distances)[:self.k]
            nearest_labels = self.y_train[nearest_indices]
            
            if self.task == 'classification':
                # Голосование для классификации
                pred = self._vote(nearest_labels)
            else:
                # Среднее для регрессии
                pred = np.mean(nearest_labels)
            
            predictions.append(pred)
        
        return np.array(predictions)

In [12]:
# Использование KNN модели

# Данные для классификации
X_train = np.array([[1, 2], [2, 3], [3, 1], [4, 2]])
y_train = np.array([0, 0, 1, 1])

X_test = np.array([[2.5, 2]])

# Обучение и предсказание
knn = KNN_Estimator(k=3, task='classification')
knn.fit(X_train, y_train)
predictions = knn.predict(X_test)
print(f"Предсказание: {predictions}")

Предсказание: [0]
